# Lab 2 — All-in-One Particle Tracking & Diffusion Analysis

**PHYS 382 Advanced Lab — Microscopy & Motility**

This notebook performs the complete analysis pipeline:
1. **Particle tracking** from `.avi` video (temporal median background, connected components, Hungarian linking)
2. **Track segmentation** (split at jumps, filter short/stationary tracks)
3. **Trajectory visualization** in physical units
4. **Displacement histogram** with Gaussian fit → D (variance & Gaussian methods)
5. **MSD analysis** with linear fit → D (MSD slope method) and power-law fit → α exponent
6. **Stokes-Einstein comparison** with viscosity adjusted for glycerol concentration
7. **Summary** with all results saved to disk

### Usage
1. Set `AVI_PATH` and experiment parameters in Cell 2
2. **Run All Cells** (Ctrl+Shift+Enter or Cell → Run All)
3. Figures auto-saved to `Analysis/figures/{date}/{filename}/`
4. Re-running deletes old figures and creates fresh ones

In [ ]:
# ============================================================================
# CELL 2 — USER INPUTS (CHANGE THESE FOR EACH VIDEO)
# ============================================================================

# Paste the full path to your .avi video file
AVI_PATH = r"D:\Documents\SFU\PHYS382-AdvancedLab\phys332w-sfu-GIT\phys332W-sfu\Lab2-Microscopy-and-Motility\Data\2026-02-24\3um-0_5p-4_9pstock-0pgly-trial1.avi"

# --- Experiment metadata (change per video) ---
BEAD_DIAMETER_UM = 3.0        # Bead diameter in micrometers
GLYCEROL_PERCENT = 0.0        # Glycerol weight percentage (0 = pure water)
PIXEL_SIZE = 0.0684           # um/px (68.4 nm/px, 100x oil — confirmed Sessions 1,3,4)
FRAME_RATE = 226              # fps (frames per second)
TEMPERATURE_C = 21.0          # Celsius

# --- Tracking parameters (usually don't need changing) ---
DETECTION_THRESHOLD = 15      # Intensity threshold above background (0-255)
GAUSSIAN_BLUR_SIGMA = 1.5     # Gaussian blur sigma for noise reduction
MIN_PARTICLE_AREA = 4         # Minimum connected component area (px^2)
MAX_PARTICLE_AREA = 200       # Maximum connected component area (px^2)
MAX_DISPLACEMENT = 10         # Maximum linking distance (px/frame)
MAX_GAP_FRAMES = 3            # Gap-closing up to N frames
MIN_TRACK_LENGTH = 10         # Minimum track length to keep (frames)
MIN_TOTAL_DISPLACEMENT = 3.0  # Minimum net displacement to keep (px)

# --- Analysis parameters ---
NUM_BEST_SEGMENTS = 10        # Number of best track segments to use
MIN_SEGMENT_LENGTH = 10       # Minimum segment length after splitting (frames)
MAX_JUMP_PX = 20              # Split tracks at jumps larger than this (px)

In [ ]:
# ============================================================================
# CELL 3 — IMPORTS & FIGURE DIRECTORY SETUP
# ============================================================================

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil
import time
from pathlib import Path
from scipy.optimize import linear_sum_assignment, curve_fit
from math import sqrt, pi

# --- Physical constants ---
k_B = 1.381e-23               # Boltzmann constant (J/K)
TEMPERATURE = TEMPERATURE_C + 273.15  # Kelvin

# --- Parse AVI path to extract date folder and filename ---
avi_path = Path(AVI_PATH)
if not avi_path.exists():
    raise FileNotFoundError(f"AVI file not found: {AVI_PATH}")

parts = avi_path.parts
idx = [i for i, p in enumerate(parts) if p.lower() == 'data']
if idx:
    date_folder = parts[idx[-1] + 1]   # e.g. "24-Feb" or "2026-02-05"
else:
    date_folder = avi_path.parent.name
file_stem = avi_path.stem              # e.g. "3um-0_5p-4_9pstock-0pGly-trail1"

# --- Set up figure output directory ---
# Figures dir is relative to this notebook's location (Analysis/)
NOTEBOOK_DIR = Path(os.getcwd())
FIGURES_DIR = NOTEBOOK_DIR / 'figures' / date_folder / file_stem

# Delete old figures and create fresh directory
if FIGURES_DIR.exists():
    shutil.rmtree(FIGURES_DIR)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# --- Viscosity calculation (Cheng 2008 formula for glycerol-water mixtures) ---
def get_viscosity(glycerol_pct, temp_c):
    """Empirical viscosity for glycerol-water mixture (Pa.s).
    Uses Cheng (2008) correlation for glycerol-water mixtures.
    Valid for 0-100% glycerol, 0-100 C.
    """
    T = temp_c
    # Water viscosity (Vogel equation, mPa.s)
    a_w = 1.790
    eta_water = a_w * np.exp((-1230 - T) * T / (36100 + 360 * T))
    
    if glycerol_pct <= 0:
        return eta_water * 1e-3  # Convert mPa.s to Pa.s
    
    # Glycerol viscosity (mPa.s)
    a_g = 12100
    eta_glycerol = a_g * np.exp((-1233 + T) * T / (9900 + 70 * T))
    
    # Cheng mixing rule
    cm = glycerol_pct / 100.0  # mass fraction
    alpha_cheng = 0.705 - 0.0017 * T
    a_mix = cm ** alpha_cheng * (1 - cm) ** (1 - alpha_cheng)
    
    ln_eta = cm * np.log(eta_glycerol) + (1 - cm) * np.log(eta_water) \
             + a_mix * cm * (1 - cm)
    
    return np.exp(ln_eta) * 1e-3  # Convert mPa.s to Pa.s

VISCOSITY = get_viscosity(GLYCEROL_PERCENT, TEMPERATURE_C)
dt = 1.0 / FRAME_RATE

# --- Print configuration summary ---
print('=' * 70)
print('LAB 2 — PARTICLE TRACKING & DIFFUSION ANALYSIS PIPELINE')
print('=' * 70)
print(f'\nVideo: {avi_path.name}')
print(f'  Date folder: {date_folder}')
print(f'  Full path: {AVI_PATH}')
print(f'\nExperiment parameters:')
print(f'  Bead diameter: {BEAD_DIAMETER_UM} um')
print(f'  Glycerol: {GLYCEROL_PERCENT}%')
print(f'  Pixel size: {PIXEL_SIZE} um/px ({PIXEL_SIZE*1000:.1f} nm/px)')
print(f'  Frame rate: {FRAME_RATE} fps (dt = {dt:.6f} s)')
print(f'  Temperature: {TEMPERATURE_C} C ({TEMPERATURE:.2f} K)')
print(f'  Viscosity: {VISCOSITY:.6f} Pa.s')
print(f'\nFigures will be saved to:')
print(f'  {FIGURES_DIR}')

In [ ]:
# ============================================================================
# CELL 4 — PARTICLE TRACKING ENGINE
# (Functions from track_onion_particles.py)
# ============================================================================

def compute_temporal_median(cap, sample_every=10):
    """Compute per-pixel temporal median from sampled frames."""
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_sampled = []
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for i in range(0, total_frames, sample_every):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) if len(frame.shape) == 3 else frame
        frames_sampled.append(gray.astype(np.float32))
    if len(frames_sampled) == 0:
        raise RuntimeError("Could not read any frames from video")
    print(f"  Sampled {len(frames_sampled)} frames for background estimation")
    background = np.median(np.stack(frames_sampled), axis=0).astype(np.uint8)
    return background


def detect_particles(gray_frame, background, threshold, blur_sigma,
                     min_area, max_area):
    """Detect particles by background subtraction + thresholding."""
    ksize = int(blur_sigma * 4) | 1
    blurred = cv2.GaussianBlur(gray_frame, (ksize, ksize), blur_sigma)
    diff = cv2.absdiff(blurred, background)
    _, binary = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    n_labels, labels, stats, centroids_cv = cv2.connectedComponentsWithStats(
        binary, connectivity=8)
    particles = []
    for label_id in range(1, n_labels):
        area = stats[label_id, cv2.CC_STAT_AREA]
        if area < min_area or area > max_area:
            continue
        mask = (labels == label_id)
        weights = diff[mask].astype(np.float64)
        weight_sum = weights.sum()
        if weight_sum > 0:
            ys, xs = np.where(mask)
            cx = np.average(xs.astype(np.float64), weights=weights)
            cy = np.average(ys.astype(np.float64), weights=weights)
        else:
            cx = centroids_cv[label_id, 0]
            cy = centroids_cv[label_id, 1]
        particles.append((cx, cy))
    return particles, binary


def track_all_frames(detections_per_frame, max_disp, max_gap, min_length):
    """Link detections across all frames into tracks (Hungarian algorithm)."""
    all_tracks = []
    active_tracks = []
    next_id = 1
    frame_numbers = sorted(detections_per_frame.keys())
    
    for frame_num in frame_numbers:
        curr_detections = detections_per_frame[frame_num]
        prev_positions = []
        prev_track_indices = []
        prev_gap_sizes = []
        
        for i, track in enumerate(active_tracks):
            gap = frame_num - track['last_seen_frame']
            if gap <= max_gap + 1:
                pos = track['positions'][track['last_seen_frame']]
                prev_positions.append(pos)
                prev_track_indices.append(i)
                prev_gap_sizes.append(gap)
        
        effective_max_disps = [max_disp * g for g in prev_gap_sizes]
        
        if len(prev_positions) > 0 and len(curr_detections) > 0:
            n_prev = len(prev_positions)
            n_curr = len(curr_detections)
            INF = 1e9
            cost = np.full((n_prev, n_curr), INF)
            for i, (px, py) in enumerate(prev_positions):
                for j, (cx, cy) in enumerate(curr_detections):
                    d = np.sqrt((px - cx)**2 + (py - cy)**2)
                    if d <= effective_max_disps[i]:
                        cost[i, j] = d
            row_ind, col_ind = linear_sum_assignment(cost)
            matches = {}
            for r, c in zip(row_ind, col_ind):
                if cost[r, c] < INF:
                    matches[r] = c
            matched_curr = set(matches.values())
            unmatched_curr = [j for j in range(n_curr) if j not in matched_curr]
        else:
            matches = {}
            unmatched_curr = list(range(len(curr_detections)))
        
        for prev_idx, curr_idx in matches.items():
            track_idx = prev_track_indices[prev_idx]
            track = active_tracks[track_idx]
            gap = frame_num - track['last_seen_frame']
            track['positions'][frame_num] = curr_detections[curr_idx]
            track['flags'][frame_num] = '*' if gap > 1 else ' '
            track['last_seen_frame'] = frame_num
            track['end_frame'] = frame_num
        
        still_active = []
        for i, track in enumerate(active_tracks):
            if frame_num - track['last_seen_frame'] > max_gap:
                all_tracks.append(track)
            else:
                still_active.append(track)
        active_tracks = still_active
        
        for curr_idx in unmatched_curr:
            new_track = {
                'id': next_id,
                'positions': {frame_num: curr_detections[curr_idx]},
                'flags': {frame_num: ' '},
                'start_frame': frame_num,
                'end_frame': frame_num,
                'last_seen_frame': frame_num,
            }
            active_tracks.append(new_track)
            next_id += 1
    
    all_tracks.extend(active_tracks)
    return all_tracks


def clean_tracks(tracks, min_length, min_displacement):
    """Remove spurious tracks (too short or stationary)."""
    cleaned = []
    for track in tracks:
        frames = sorted(track['positions'].keys())
        if len(frames) < min_length:
            continue
        first = track['positions'][frames[0]]
        last = track['positions'][frames[-1]]
        net_disp = np.sqrt((last[0] - first[0])**2 + (last[1] - first[1])**2)
        if net_disp < min_displacement:
            continue
        cleaned.append(track)
    for i, track in enumerate(cleaned, 1):
        track['id'] = i
    return cleaned


def write_mtrack2_format(tracks, total_frames, output_path):
    """Write tracks in MTrack2-compatible tab-separated format."""
    n_tracks = len(tracks)
    with open(output_path, 'w') as f:
        header_parts = ['Frame']
        for i in range(1, n_tracks + 1):
            header_parts.extend([f'X{i}', f'Y{i}', f'Flag{i}'])
        f.write('\t'.join(header_parts) + '\n')
        f.write(f'Tracks 1 to {n_tracks}\n')
        for frame in range(1, total_frames + 1):
            row_parts = [str(frame)]
            for track in tracks:
                if frame in track['positions']:
                    x, y = track['positions'][frame]
                    flag = track['flags'].get(frame, ' ')
                    row_parts.extend([f'{x:.5f}', f'{y:.5f}', flag])
                else:
                    row_parts.extend([' ', ' ', ' '])
            f.write('\t'.join(row_parts) + '\n')
        f.write('\n')
        f.write('Track \tLength\tDistance traveled\tNr of Frames\n')
        for i, track in enumerate(tracks, 1):
            frames = sorted(track['positions'].keys())
            path_length = 0.0
            for j in range(1, len(frames)):
                p1 = track['positions'][frames[j - 1]]
                p2 = track['positions'][frames[j]]
                path_length += np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
            start = track['positions'][frames[0]]
            end = track['positions'][frames[-1]]
            distance = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            f.write(f'{i}\t{path_length:.5f}\t{distance:.5f}\t{len(frames)}\n')


# ======================== RUN TRACKING PIPELINE ========================

print('\n' + '=' * 70)
print('STEP 1: PARTICLE TRACKING')
print('=' * 70)

cap = cv2.VideoCapture(AVI_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open video: {AVI_PATH}")

fps_meta = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f'\nVideo info:')
print(f'  Resolution: {width} x {height}')
print(f'  Total frames: {total_frames}')
print(f'  Metadata FPS: {fps_meta}')
print(f'  Using FRAME_RATE: {FRAME_RATE} fps')
print(f'  Duration: {total_frames / FRAME_RATE:.2f} s')

# --- Background estimation ---
print(f'\nComputing temporal median background...')
t0 = time.time()
background = compute_temporal_median(cap, sample_every=10)
print(f'  Done in {time.time() - t0:.1f} s')

# --- Detect particles per frame ---
print(f'\nDetecting particles in {total_frames} frames...')
t0 = time.time()
detections_per_frame = {}
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

for frame_num in range(1, total_frames + 1):
    ret, frame = cap.read()
    if not ret:
        total_frames = frame_num - 1
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) if len(frame.shape) == 3 else frame
    particles, _ = detect_particles(gray, background, DETECTION_THRESHOLD,
                                     GAUSSIAN_BLUR_SIGMA, MIN_PARTICLE_AREA,
                                     MAX_PARTICLE_AREA)
    detections_per_frame[frame_num] = particles
    if frame_num % 500 == 0 or frame_num == 1:
        print(f'  Frame {frame_num}/{total_frames}: {len(particles)} particles')

cap.release()
elapsed = time.time() - t0
print(f'  Detection complete in {elapsed:.1f} s ({total_frames / max(elapsed, 0.01):.0f} fps)')

counts = [len(detections_per_frame.get(f, [])) for f in range(1, total_frames + 1)]
print(f'  Particles per frame: min={min(counts)}, max={max(counts)}, mean={np.mean(counts):.1f}')

# --- Link into tracks ---
print(f'\nLinking particles into tracks...')
t0 = time.time()
raw_tracks = track_all_frames(detections_per_frame, MAX_DISPLACEMENT, MAX_GAP_FRAMES, 1)
print(f'  Raw tracks: {len(raw_tracks)}')

tracks = clean_tracks(raw_tracks, MIN_TRACK_LENGTH, MIN_TOTAL_DISPLACEMENT)
print(f'  Valid tracks after cleaning: {len(tracks)}')
print(f'  Tracking complete in {time.time() - t0:.1f} s')

if len(tracks) == 0:
    raise RuntimeError("No valid tracks found! Try adjusting DETECTION_THRESHOLD, "
                       "MIN_TRACK_LENGTH, or MAX_DISPLACEMENT.")

# --- Save MTrack2 format ---
track_output = str(FIGURES_DIR / 'trackresults.txt')
write_mtrack2_format(tracks, total_frames, track_output)
print(f'\nTrack data saved to: {track_output}')

# --- Track statistics ---
lengths = [len(t['positions']) for t in tracks]
print(f'\nTrack statistics:')
print(f'  Track length (frames): min={min(lengths)}, max={max(lengths)}, mean={np.mean(lengths):.1f}')

In [ ]:
# ============================================================================
# CELL 5 — LOAD & SEGMENT TRACKS
# (Functions from Diffusion_Analysis_Corrected.ipynb)
# ============================================================================

def load_mtrack2_data(file_path):
    """Load MTrack2 output file and return cleaned data matrix."""
    my_data = np.genfromtxt(file_path, delimiter='\t', skip_header=2,
                           skip_footer=1, invalid_raise=False)
    # Remove Frame column and Flag columns (keep only X, Y pairs)
    A = np.zeros(my_data.shape[1] // 3 + 1, dtype=int)
    for i in range(my_data.shape[1] // 3 + 1):
        A[i] = 3 * i
    new_data = np.delete(my_data, A, axis=1)
    # Sort data — move NaN values to the end
    mask = np.isnan(new_data)
    new_mask = np.zeros(mask.shape)
    for ind, value in enumerate(mask):
        new_mask[ind, :] = ~value * (ind + 1)
    new_mask = new_mask.astype(np.int_)
    for row_index, row in enumerate(new_mask):
        for col_index, item in enumerate(row):
            if item == 0:
                new_mask[row_index][col_index] = new_mask.shape[0] + 5
    for i in range(new_mask.shape[1]):
        new_mask[:, i] = np.sort(new_mask[:, i])
    data = np.empty((mask.shape[0], mask.shape[1]))
    data[:, :] = np.nan
    for i in range(new_mask.shape[0]):
        for j in range(new_mask.shape[1]):
            temp = new_mask[i, j]
            if temp < new_mask.shape[0]:
                data[i, j] = new_data[temp, j]
    return data


def split_tracks_at_jumps(data, min_length, max_jump_px):
    """Extract valid trajectory segments, splitting at large jumps."""
    segments = []
    n_particles = data.shape[1] // 2
    segment_id = 0
    for i in range(n_particles):
        x_col = i * 2
        y_col = i * 2 + 1
        x_raw = data[:, x_col]
        y_raw = data[:, y_col]
        valid_mask = ~np.isnan(x_raw) & ~np.isnan(y_raw)
        x_clean = x_raw[valid_mask]
        y_clean = y_raw[valid_mask]
        if len(x_clean) < min_length:
            continue
        dx = np.diff(x_clean)
        dy = np.diff(y_clean)
        steps = np.sqrt(dx**2 + dy**2)
        bad_jump_indices = np.where(steps > max_jump_px)[0]
        if len(bad_jump_indices) == 0:
            segment_id += 1
            segments.append({'x': x_clean, 'y': y_clean, 'id': segment_id,
                             'original_particle': i + 1, 'length': len(x_clean)})
        else:
            split_indices = bad_jump_indices + 1
            x_segments = np.split(x_clean, split_indices)
            y_segments = np.split(y_clean, split_indices)
            for x_seg, y_seg in zip(x_segments, y_segments):
                if len(x_seg) >= min_length:
                    segment_id += 1
                    segments.append({'x': x_seg, 'y': y_seg, 'id': segment_id,
                                     'original_particle': i + 1, 'length': len(x_seg)})
    segments.sort(key=lambda s: s['length'], reverse=True)
    return segments


# ======================== LOAD & SEGMENT ========================

print('\n' + '=' * 70)
print('STEP 2: LOAD & SEGMENT TRACKS')
print('=' * 70)

data = load_mtrack2_data(track_output)
print(f'\nLoaded data: {data.shape[0]} frames, {data.shape[1] // 2} particles')

segments = split_tracks_at_jumps(data, MIN_SEGMENT_LENGTH, MAX_JUMP_PX)
print(f'Valid segments after splitting: {len(segments)}')

if len(segments) == 0:
    raise RuntimeError("No valid segments! Try lowering MIN_SEGMENT_LENGTH or increasing MAX_JUMP_PX.")

n_to_use = min(NUM_BEST_SEGMENTS, len(segments))
selected = segments[:n_to_use]

print(f'\nUsing top {n_to_use} segments (by length):')
print(f'  {"Seg":<5} {"Orig Part":<10} {"Frames":<8}')
print(f'  {"-"*25}')
for seg in selected[:10]:
    print(f'  {seg["id"]:<5} {seg["original_particle"]:<10} {seg["length"]:<8}')

In [ ]:
# ============================================================================
# CELL 6 — PLOT 1: TRAJECTORIES
# ============================================================================

print('\n' + '=' * 70)
print('STEP 3: TRAJECTORY PLOT')
print('=' * 70)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
colors = plt.cm.tab10(np.linspace(0, 1, n_to_use))

# Left: absolute positions (pixels)
ax = axes[0]
for seg, c in zip(selected, colors):
    ax.plot(seg['x'], seg['y'], '-', linewidth=1, color=c, alpha=0.7)
    ax.plot(seg['x'][0], seg['y'][0], 'o', color=c, markersize=4)
    ax.plot(seg['x'][-1], seg['y'][-1], 's', color=c, markersize=4)
ax.set_xlabel('X (pixels)')
ax.set_ylabel('Y (pixels)')
ax.set_title(f'Top {n_to_use} Trajectories — Absolute Position')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

# Right: displacement from start (microns)
ax = axes[1]
for seg, c in zip(selected, colors):
    x_um = (seg['x'] - seg['x'][0]) * PIXEL_SIZE
    y_um = (seg['y'] - seg['y'][0]) * PIXEL_SIZE
    ax.plot(x_um, y_um, '-o', markersize=1, linewidth=1, color=c, alpha=0.7)
ax.set_xlabel(r'$\Delta X$ ($\mu$m)')
ax.set_ylabel(r'$\Delta Y$ ($\mu$m)')
ax.set_title(f'Displacement from Start ({PIXEL_SIZE*1000:.1f} nm/px)')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

plt.tight_layout()
fig.savefig(str(FIGURES_DIR / 'trajectories.png'), dpi=300, bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR / "trajectories.png"}')

In [ ]:
# ============================================================================
# CELL 7 — PLOT 2: DISPLACEMENT HISTOGRAM + GAUSSIAN FIT
# ============================================================================

print('\n' + '=' * 70)
print('STEP 4: DISPLACEMENT ANALYSIS')
print('=' * 70)

# --- Helper functions ---
def gaussian(x, amplitude, mean, std_dev):
    return amplitude * np.exp(-(x - mean)**2 / (2 * std_dev**2))

def compute_chi_squared(observed, expected, errors, n_params):
    """Compute chi-squared and reduced chi-squared."""
    valid = errors > 0
    if np.sum(valid) <= n_params:
        return np.nan, np.nan, 0, np.nan
    residuals = (observed[valid] - expected[valid]) / errors[valid]
    chi2 = np.sum(residuals**2)
    dof = np.sum(valid) - n_params
    chi2_red = chi2 / dof if dof > 0 else np.nan
    from scipy.stats import chi2 as chi2_dist
    pval = 1 - chi2_dist.cdf(chi2, dof) if dof > 0 else np.nan
    return chi2, chi2_red, dof, pval

# --- Extract displacements ---
all_dx = []
all_dy = []
for seg in selected:
    all_dx.extend(np.diff(seg['x']))
    all_dy.extend(np.diff(seg['y']))

dx_px = np.array(all_dx)
dy_px = np.array(all_dy)
dx_um = dx_px * PIXEL_SIZE
dy_um = dy_px * PIXEL_SIZE
n_steps = len(dx_px)

print(f'\nTotal displacement steps: {n_steps}')

# --- METHOD 1: Direct Variance ---
var_dx = np.var(dx_px)  # px^2
var_dy = np.var(dy_px)
D_x_direct = var_dx * PIXEL_SIZE**2 / (2 * dt)  # um^2/s
D_y_direct = var_dy * PIXEL_SIZE**2 / (2 * dt)
D_direct = (D_x_direct + D_y_direct) / 2
D_direct_err = abs(D_x_direct - D_y_direct) / 2

print(f'\nMETHOD 1: Direct Variance')
print(f'  D_x = {D_x_direct:.4f} um^2/s')
print(f'  D_y = {D_y_direct:.4f} um^2/s')
print(f'  D_avg = {D_direct:.4f} +/- {D_direct_err:.4f} um^2/s')

# --- METHOD 2: Gaussian Fit ---
nbins = 20

# X direction
counts_x, bin_edges_x = np.histogram(dx_um, bins=nbins)
bin_centers_x = (bin_edges_x[:-1] + bin_edges_x[1:]) / 2
counts_err_x = np.sqrt(np.maximum(counts_x, 1))

try:
    popt_x, pcov_x = curve_fit(gaussian, bin_centers_x, counts_x,
                                p0=[max(counts_x), 0, np.std(dx_um)])
    std_x_fit = abs(popt_x[2])
    std_x_fit_err = np.sqrt(pcov_x[2, 2]) if pcov_x[2, 2] > 0 else 0
    gauss_pred_x = gaussian(bin_centers_x, *popt_x)
    chi2_x, chi2_red_x, dof_x, pval_x = compute_chi_squared(
        counts_x, gauss_pred_x, counts_err_x, 3)
except Exception:
    std_x_fit = np.std(dx_um)
    std_x_fit_err = 0
    popt_x = [max(counts_x), 0, std_x_fit]

# Y direction
counts_y, bin_edges_y = np.histogram(dy_um, bins=nbins)
bin_centers_y = (bin_edges_y[:-1] + bin_edges_y[1:]) / 2
counts_err_y = np.sqrt(np.maximum(counts_y, 1))

try:
    popt_y, pcov_y = curve_fit(gaussian, bin_centers_y, counts_y,
                                p0=[max(counts_y), 0, np.std(dy_um)])
    std_y_fit = abs(popt_y[2])
    std_y_fit_err = np.sqrt(pcov_y[2, 2]) if pcov_y[2, 2] > 0 else 0
    gauss_pred_y = gaussian(bin_centers_y, *popt_y)
    chi2_y, chi2_red_y, dof_y, pval_y = compute_chi_squared(
        counts_y, gauss_pred_y, counts_err_y, 3)
except Exception:
    std_y_fit = np.std(dy_um)
    std_y_fit_err = 0
    popt_y = [max(counts_y), 0, std_y_fit]

D_x_fit = std_x_fit**2 / (2 * dt)
D_y_fit = std_y_fit**2 / (2 * dt)
D_fit = (D_x_fit + D_y_fit) / 2
D_fit_err = abs(D_x_fit - D_y_fit) / 2

print(f'\nMETHOD 2: Gaussian Fit')
print(f'  sigma_x = {std_x_fit:.4f} um, sigma_y = {std_y_fit:.4f} um')
print(f'  D_fit = {D_fit:.4f} +/- {D_fit_err:.4f} um^2/s')

# --- Plot histogram ---
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# X histogram
ax = axes[0]
ax.bar(bin_centers_x, counts_x, width=bin_edges_x[1]-bin_edges_x[0],
       alpha=0.6, color='steelblue', label='Data')
x_fine = np.linspace(bin_centers_x[0], bin_centers_x[-1], 200)
ax.plot(x_fine, gaussian(x_fine, *popt_x), 'r-', linewidth=2,
        label=f'Gaussian fit\n$\\sigma$ = {std_x_fit:.4f} $\\mu$m')
ax.set_xlabel(r'$\Delta x$ per frame ($\mu$m)')
ax.set_ylabel('Count')
ax.set_title(f'X Displacement — D_x = {D_x_fit:.4f} $\\mu$m$^2$/s')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Y histogram
ax = axes[1]
ax.bar(bin_centers_y, counts_y, width=bin_edges_y[1]-bin_edges_y[0],
       alpha=0.6, color='darkorange', label='Data')
y_fine = np.linspace(bin_centers_y[0], bin_centers_y[-1], 200)
ax.plot(y_fine, gaussian(y_fine, *popt_y), 'r-', linewidth=2,
        label=f'Gaussian fit\n$\\sigma$ = {std_y_fit:.4f} $\\mu$m')
ax.set_xlabel(r'$\Delta y$ per frame ($\mu$m)')
ax.set_ylabel('Count')
ax.set_title(f'Y Displacement — D_y = {D_y_fit:.4f} $\\mu$m$^2$/s')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
fig.savefig(str(FIGURES_DIR / 'displacement_histogram.png'), dpi=300, bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR / "displacement_histogram.png"}')

In [ ]:
# ============================================================================
# CELL 8 — PLOT 3: MSD ANALYSIS + ALPHA EXPONENT
# ============================================================================

print('\n' + '=' * 70)
print('STEP 5: MSD ANALYSIS')
print('=' * 70)

def linear(t, slope, intercept):
    return slope * t + intercept

def power_law(t, K, alpha):
    return K * t**alpha

# --- Compute ensemble MSD ---
min_track = min([seg['length'] for seg in selected])
max_lag = min(min_track // 2, 30)

all_MSDs = []
for seg in selected:
    x = seg['x'].copy()
    y = seg['y'].copy()
    MSD_seg = np.zeros(max_lag)
    n_frames = len(x)
    for lag in range(max_lag):
        dx_lag = x[lag+1:] - x[:n_frames-lag-1]
        dy_lag = y[lag+1:] - y[:n_frames-lag-1]
        r_sq = dx_lag**2 + dy_lag**2
        MSD_seg[lag] = np.mean(r_sq) if len(r_sq) > 0 else 0
    all_MSDs.append(MSD_seg)

all_MSDs = np.array(all_MSDs)
MSD_px = np.mean(all_MSDs, axis=0)
MSD_err_px = np.std(all_MSDs, axis=0) / np.sqrt(len(selected))

MSD_um = MSD_px * PIXEL_SIZE**2
MSD_err_um = MSD_err_px * PIXEL_SIZE**2
lag_times = np.arange(max_lag) * dt

# --- FIT 1: Linear fit for D ---
n_fit = max_lag // 4
fit_times = lag_times[1:n_fit+1]
fit_MSD = MSD_um[1:n_fit+1]
fit_err = MSD_err_um[1:n_fit+1]
fit_err = np.where(fit_err > 0, fit_err, 1e-10)

try:
    popt_msd, pcov_msd = curve_fit(linear, fit_times, fit_MSD,
                                    sigma=fit_err, absolute_sigma=True, p0=[1, 0])
    perr_msd = np.sqrt(np.diag(pcov_msd))
    slope = popt_msd[0]
    slope_err = perr_msd[0]
    intercept = popt_msd[1]
    msd_pred = linear(fit_times, *popt_msd)
    chi2_msd, chi2_red_msd, dof_msd, pval_msd = compute_chi_squared(
        fit_MSD, msd_pred, fit_err, 2)
except Exception:
    coeffs = np.polyfit(fit_times, fit_MSD, 1)
    slope = coeffs[0]
    slope_err = 0
    intercept = coeffs[1]
    msd_pred = linear(fit_times, slope, intercept)
    chi2_red_msd = np.nan
    pval_msd = np.nan

D_msd = slope / 4  # For 2D: MSD = 4*D*t
D_msd_err = slope_err / 4

print(f'\nMETHOD 3: MSD Slope')
print(f'  Slope = {slope:.6f} +/- {slope_err:.6f} um^2/s')
print(f'  D_MSD = {D_msd:.4f} +/- {D_msd_err:.4f} um^2/s')

# --- FIT 2: Power-law for alpha ---
n_fit_power = max_lag // 2
pl_t = lag_times[1:n_fit_power+1]
pl_msd = MSD_um[1:n_fit_power+1]
pl_err = MSD_err_um[1:n_fit_power+1]
pl_err = np.where(pl_err > 0, pl_err, 1e-10)

try:
    popt_pl, pcov_pl = curve_fit(power_law, pl_t, pl_msd,
                                  sigma=pl_err, absolute_sigma=True,
                                  p0=[0.001, 1.0], maxfev=5000)
    perr_pl = np.sqrt(np.diag(pcov_pl))
    K_fit = popt_pl[0]
    alpha_fit = popt_pl[1]
    K_err = perr_pl[0]
    alpha_err = perr_pl[1]
except Exception:
    log_t = np.log(pl_t)
    log_msd = np.log(pl_msd)
    coeffs = np.polyfit(log_t, log_msd, 1)
    alpha_fit = coeffs[0]
    K_fit = np.exp(coeffs[1])
    alpha_err = 0
    K_err = 0

# --- Classify motion ---
if alpha_fit > 1.7:
    motion_type = 'BALLISTIC / DIRECTED'
elif alpha_fit > 1.2:
    motion_type = 'SUPERDIFFUSIVE (directed + random)'
elif alpha_fit > 0.8:
    motion_type = 'DIFFUSIVE (Brownian-like)'
else:
    motion_type = 'SUBDIFFUSIVE (confined)'

print(f'\nPower-law fit: MSD = K * t^alpha')
print(f'  alpha = {alpha_fit:.3f} +/- {alpha_err:.3f}')
print(f'  Motion classification: {motion_type}')

# --- Plot MSD ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: linear scale
ax = axes[0]
ax.errorbar(lag_times, MSD_um, yerr=MSD_err_um, fmt='o',
            markersize=4, capsize=3, alpha=0.6, color='steelblue', label='MSD data')
fit_line_t = np.linspace(0, lag_times[n_fit], 100)
ax.plot(fit_line_t, linear(fit_line_t, slope, intercept), 'r-', linewidth=2,
        label=f'Linear fit: D = {D_msd:.4f} $\\pm$ {D_msd_err:.4f} $\\mu$m$^2$/s')
pl_line_t = np.linspace(dt, lag_times[n_fit_power], 100)
ax.plot(pl_line_t, power_law(pl_line_t, K_fit, alpha_fit), 'g--', linewidth=2,
        label=f'Power law: $\\alpha$ = {alpha_fit:.2f}')
ax.set_xlabel('Lag time $\\tau$ (s)', fontsize=12)
ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=12)
ax.set_title('MSD — Linear Scale')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Right: log-log scale
ax = axes[1]
valid = MSD_um[1:] > 0
ax.errorbar(lag_times[1:][valid], MSD_um[1:][valid], yerr=MSD_err_um[1:][valid],
            fmt='o', markersize=4, capsize=3, alpha=0.6, color='steelblue', label='MSD data')
ref_t = np.logspace(np.log10(dt), np.log10(lag_times[-1]), 50)
msd_at_1 = MSD_um[1] if MSD_um[1] > 0 else 1e-6
ax.plot(ref_t, msd_at_1 * (ref_t/dt)**1, 'k:', alpha=0.4, linewidth=1.5,
        label=r'$\alpha=1$ (diffusive)')
ax.plot(ref_t, msd_at_1 * (ref_t/dt)**2, 'k--', alpha=0.4, linewidth=1.5,
        label=r'$\alpha=2$ (ballistic)')
ax.plot(pl_line_t, power_law(pl_line_t, K_fit, alpha_fit), 'g-', linewidth=2,
        label=f'Fit: $\\alpha$ = {alpha_fit:.2f} $\\pm$ {alpha_err:.2f}')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Lag time $\\tau$ (s)', fontsize=12)
ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=12)
ax.set_title('MSD — Log-Log Scale')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
fig.savefig(str(FIGURES_DIR / 'msd_analysis.png'), dpi=300, bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR / "msd_analysis.png"}')

In [ ]:
# ============================================================================
# CELL 9 — PLOT 4: D COMPARISON (MEASURED vs STOKES-EINSTEIN)
# ============================================================================

print('\n' + '=' * 70)
print('STEP 6: STOKES-EINSTEIN COMPARISON')
print('=' * 70)

# --- Stokes-Einstein theoretical D ---
r_m = (BEAD_DIAMETER_UM / 2) * 1e-6  # radius in meters
D_theory = k_B * TEMPERATURE / (6 * pi * VISCOSITY * r_m)  # m^2/s
D_theory_um = D_theory * 1e12  # um^2/s

print(f'\nStokes-Einstein prediction:')
print(f'  D_theory = k_B*T / (6*pi*eta*r)')
print(f'  = {k_B:.3e} * {TEMPERATURE:.2f} / (6*pi * {VISCOSITY:.6f} * {r_m:.2e})')
print(f'  = {D_theory_um:.4f} um^2/s')

print(f'\n{"Method":<20} {"D (um^2/s)":<18} {"Deviation from theory":<22}')
print('-' * 60)
for name, D_val, D_err in [('Direct Variance', D_direct, D_direct_err),
                             ('Gaussian Fit', D_fit, D_fit_err),
                             ('MSD Slope', D_msd, D_msd_err)]:
    dev = (D_val - D_theory_um) / D_theory_um * 100
    print(f'{name:<20} {D_val:.4f} +/- {D_err:.4f}   {dev:+.1f}%')
print(f'{"Stokes-Einstein":<20} {D_theory_um:.4f}             (theory)')

# --- Bar chart ---
fig, ax = plt.subplots(figsize=(8, 6))
methods = ['Direct\nVariance', 'Gaussian\nFit', 'MSD\nSlope', 'Stokes-\nEinstein']
D_vals = [D_direct, D_fit, D_msd, D_theory_um]
D_errs = [D_direct_err, D_fit_err, D_msd_err, 0]
bar_colors = ['steelblue', 'darkorange', 'forestgreen', 'crimson']

bars = ax.bar(methods, D_vals, yerr=D_errs, capsize=5,
              color=bar_colors, alpha=0.8, edgecolor='black', linewidth=0.5)

# Add value labels on bars
for bar, val in zip(bars, D_vals):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + max(D_vals) * 0.02,
            f'{val:.4f}', ha='center', va='bottom', fontsize=9)

ax.set_ylabel(r'Diffusion Coefficient $D$ ($\mu$m$^2$/s)', fontsize=12)
ax.set_title(f'{BEAD_DIAMETER_UM} $\\mu$m beads — {GLYCEROL_PERCENT}% glycerol — '
             f'{PIXEL_SIZE*1000:.1f} nm/px', fontsize=13)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, max(D_vals) * 1.3)

plt.tight_layout()
fig.savefig(str(FIGURES_DIR / 'D_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()
print(f'Saved: {FIGURES_DIR / "D_comparison.png"}')

In [ ]:
# ============================================================================
# CELL 10 — SUMMARY
# ============================================================================

summary_lines = []
summary_lines.append('=' * 70)
summary_lines.append('ANALYSIS SUMMARY')
summary_lines.append('=' * 70)
summary_lines.append(f'')
summary_lines.append(f'Video: {avi_path.name}')
summary_lines.append(f'Date: {date_folder}')
summary_lines.append(f'Bead diameter: {BEAD_DIAMETER_UM} um')
summary_lines.append(f'Glycerol: {GLYCEROL_PERCENT}%')
summary_lines.append(f'Temperature: {TEMPERATURE_C} C ({TEMPERATURE:.2f} K)')
summary_lines.append(f'Viscosity: {VISCOSITY:.6f} Pa.s')
summary_lines.append(f'Pixel size: {PIXEL_SIZE} um/px ({PIXEL_SIZE*1000:.1f} nm/px)')
summary_lines.append(f'Frame rate: {FRAME_RATE} fps')
summary_lines.append(f'')
summary_lines.append(f'Tracking:')
summary_lines.append(f'  Total frames: {total_frames}')
summary_lines.append(f'  Valid tracks: {len(tracks)}')
summary_lines.append(f'  Segments used: {n_to_use}')
summary_lines.append(f'  Total displacement steps: {n_steps}')
summary_lines.append(f'')
summary_lines.append(f'Diffusion Coefficients (um^2/s):')
summary_lines.append(f'  Method 1 (Direct Variance): {D_direct:.4f} +/- {D_direct_err:.4f}')
summary_lines.append(f'  Method 2 (Gaussian Fit):     {D_fit:.4f} +/- {D_fit_err:.4f}')
summary_lines.append(f'  Method 3 (MSD Slope):        {D_msd:.4f} +/- {D_msd_err:.4f}')
summary_lines.append(f'  Stokes-Einstein (theory):    {D_theory_um:.4f}')
summary_lines.append(f'')
summary_lines.append(f'Deviations from Stokes-Einstein:')
for name, D_val in [('Direct Variance', D_direct),
                     ('Gaussian Fit', D_fit),
                     ('MSD Slope', D_msd)]:
    dev = (D_val - D_theory_um) / D_theory_um * 100
    summary_lines.append(f'  {name}: {dev:+.1f}%')
summary_lines.append(f'')
summary_lines.append(f'MSD Exponent:')
summary_lines.append(f'  alpha = {alpha_fit:.3f} +/- {alpha_err:.3f}')
summary_lines.append(f'  Classification: {motion_type}')
summary_lines.append(f'')
summary_lines.append(f'Figures saved to: {FIGURES_DIR}')
summary_lines.append('=' * 70)

summary_text = '\n'.join(summary_lines)

# Print to notebook
print(summary_text)

# Save to file
with open(str(FIGURES_DIR / 'summary.txt'), 'w') as f:
    f.write(summary_text)

print(f'\nSummary saved to: {FIGURES_DIR / "summary.txt"}')